## Model Fit in Linear Regression - Lab

## Introduction
In this lab, you'll learn how to evaluate your model results, and you'll learn methods to select the appropriate features using stepwise selection.

## Objectives
You will be able to:
* Analyze the results of regression and R-squared and adjusted-R-squared 
* Understand and apply forward and backward predictor selection

## The Boston Housing Data once more

We pre-processed the Boston Housing Data the same way we did before:

- We dropped "ZN" and "NOX" completely
- We categorized "RAD" in 3 bins and "TAX" in 4 bins
- We used min-max-scaling on "B", "CRIM" and "DIS" (and logtransformed all of them first, except "B")
- We used standardization on "AGE", "INDUS", "LSTAT" and "PTRATIO" (and logtransformed all of them first, except for "AGE") 

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
boston_features = boston_features.drop(["NOX","ZN"],axis=1)

# first, create bins for based on the values observed. 3 values will result in 2 bins
bins = [0,6,  24]
bins_rad = pd.cut(boston_features['RAD'], bins)
bins_rad = bins_rad.cat.as_unordered()

# first, create bins for based on the values observed. 4 values will result in 3 bins
bins = [0, 270, 360, 712]
bins_tax = pd.cut(boston_features['TAX'], bins)
bins_tax = bins_tax.cat.as_unordered()

tax_dummy = pd.get_dummies(bins_tax, prefix="TAX")
rad_dummy = pd.get_dummies(bins_rad, prefix="RAD")
boston_features = boston_features.drop(["RAD","TAX"], axis=1)
boston_features = pd.concat([boston_features, rad_dummy, tax_dummy], axis=1)

age = boston_features["AGE"]
b = boston_features["B"]
logcrim = np.log(boston_features["CRIM"])
logdis = np.log(boston_features["DIS"])
logindus = np.log(boston_features["INDUS"])
loglstat = np.log(boston_features["LSTAT"])
logptratio = np.log(boston_features["PTRATIO"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["CRIM"] = (logcrim-min(logcrim))/(max(logcrim)-min(logcrim))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["AGE"] = (age-np.mean(age))/np.sqrt(np.var(age))
boston_features["INDUS"] = (logindus-np.mean(logindus))/np.sqrt(np.var(logindus))
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))
boston_features["PTRATIO"] = (logptratio-np.mean(logptratio))/(np.sqrt(np.var(logptratio)))

## Perform stepwise selection

The code for stepwise selection is copied below.

In [3]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [25]:
boston_features = boston_features.drop(['RAD_(6, 24]', 'TAX_(360, 712]'], axis = 1)
predictors = sm.add_constant(boston_features)
result = stepwise_selection(predictors, boston.target)

Add  const                          with p-value 9.37062e-216
Add  LSTAT                          with p-value 9.27989e-122
Add  RM                             with p-value 1.98621e-16
Add  PTRATIO                        with p-value 2.5977e-12
Add  DIS                            with p-value 2.85496e-09
Add  B                              with p-value 2.77572e-06
Add  TAX_(0, 270]                   with p-value 0.000855799
Add  CHAS                           with p-value 0.00151282
Add  INDUS                          with p-value 0.00588575


### Build the final model again in Statsmodels

In [26]:
model = sm.OLS(boston.target, predictors).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.779
Model:                            OLS   Adj. R-squared:                  0.774
Method:                 Least Squares   F-statistic:                     144.9
Date:                Wed, 07 Nov 2018   Prob (F-statistic):          5.15e-153
Time:                        21:40:15   Log-Likelihood:                -1458.2
No. Observations:                 506   AIC:                             2942.
Df Residuals:                     493   BIC:                             2997.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              7.7911      3.406      2.288      0.023       1.100      14.482
CRIM              -1.9000      2.091     -0.909      0.364      -6.009       2.209
INDUS             -0.8069      0.362     -2.228      0.026      -1.518      -0.095
CHAS               2.5968      0.796      3.262      0.001       1.033       4.161
RM                 2.6445      0.408      6.480      0.000       1.843       3.446
AGE                0.0787      0.352      0.224      0.823      -0.612       0.770
DIS              -10.0839      1.855     -5.437      0.000     -13.728      -6.440
PTRATIO           -1.4864      0.241     -6.159      0.000      -1.961      -1.012
B                  3.8623      0.981      3.935      0.000       1.934       5.791
LSTAT             -5.6315      0.354    -15.929      0.000      -6.326      -4.937
RAD_(0, 6]        -1.3271      0.669     -1.983      0.048      -2.642      -0.012
TAX_(0, 270]       2.1487      0.704      3.051      0.002       0.765       3.532
TAX_(270, 360]     0.8874      0.607      1.461      0.145      -0.306       2.081
==============================================================================
Omnibus:                      106.736   Durbin-Watson:                   1.093
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              431.931
Skew:                           0.891   Prob(JB):                     1.61e-94
Kurtosis:                       7.161   Cond. No.                         126.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Where our stepwise procedure mentions that "CHAS" was added with a p-value of 0.00151282, but our statsmodels output returns a p-value of 0.000. What is the intuition behind this?

## Use Feature ranking with recursive feature elimination

Use feature ranking to select the 5 most important features

In [27]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

lin = LinearRegression()
selector = RFE(lin, n_features_to_select = 5)
selector = selector.fit(predictors, boston.target)

In [28]:
selector.support_

array([False, False, False,  True,  True, False,  True, False,  True,
        True, False, False, False])

In [22]:
estimator = selector.estimator_
print(estimator.intercept_)
estimator.coef_


-0.49739821537892226


array([ 2.93498961,  3.43718997, -6.58036332,  4.65357304, -6.25217488])

Fit the linear regression model again using the 5 columns selected

In [31]:
chosen_five = []
count = 0
for column in boston_features:
    if selector.support_[count]:
        chosen_five.append(column)
    count +=1
model2 = sm.OLS(boston.target, boston_features[chosen_five]).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.963
Model:                            OLS   Adj. R-squared:                  0.963
Method:                 Least Squares   F-statistic:                     2626.
Date:                Wed, 07 Nov 2018   Prob (F-statistic):               0.00
Time:                        21:42:56   Log-Likelihood:                -1497.3
No. Observations:                 506   AIC:                             3005.
Df Residuals:                     501   BIC:                             3026.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
RM             3.5841      0.060     59.689      0.000       3.466       3.702
AGE            1.0408      0.270      3.853      0.000       0.510       1.572
PTRATIO       -1.6423      0.240     -6.829      0.000      -2.115      -1.170
LSTAT         -5.7949      0.280    -20.706      0.000      -6.345      -5.245
RAD_(0, 6]    -0.0409      0.487     -0.084      0.933      -0.997       0.915
==============================================================================
Omnibus:                      154.963   Durbin-Watson:                   0.996
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              714.635
Skew:                           1.285   Prob(JB):                    6.59e-156
Kurtosis:                       8.224   Cond. No.                         15.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Now, predict $\hat y$ using your model. you can use `.predict()` in scikit-learn

In [32]:
model2.predict()

array([33.16145372, 25.26137344, 35.24598912, 36.40412127, 31.50582197,
       29.32237147, 22.45194191, 19.94511188, 13.83961847, 20.05846867,
       19.97920019, 22.41320156, 18.72890857, 21.73662778, 21.01050706,
       20.88593665, 22.67868631, 17.07535848, 15.68425178, 18.21786246,
       12.70495656, 17.83254622, 15.63686682, 14.18449778, 16.2806512 ,
       14.68134201, 16.66757274, 15.96172926, 20.67171904, 21.68962126,
       12.36980609, 19.18356722, 10.80362468, 14.37111171, 14.86324884,
       21.56500669, 19.39629596, 21.23236771, 19.83822162, 30.6411545 ,
       39.48788551, 29.76274434, 25.98227068, 23.74190414, 22.06264931,
       19.80101846, 17.00489533, 17.06844683, 10.40094355, 16.10155275,
       19.4232993 , 24.03614449, 29.50340103, 23.15928942, 15.29996787,
       32.24277199, 28.3420256 , 35.52119706, 23.90787299, 20.93852334,
       17.54766467, 18.45741983, 26.63585164, 23.50183149, 27.63817273,
       30.33483191, 21.44494243, 21.56340401, 16.48198317, 21.22

Now, using the formulas of R-squared and adjusted-R-squared below, and your Python/numpy knowledge, compute them and contrast them with the R-squared and adjusted-R-squared in your statsmodels output using stepwise selection. Which of the two models would you prefer?

$SS_{residual} = \sum (y - \hat{y})^2 $

$SS_{total} = \sum (y - \bar{y})^2 $

$R^2 = 1- \dfrac{SS_{residual}}{SS_{total}}$

$R^2_{adj}= 1-(1-R^2)\dfrac{n-1}{n-p-1}$

In [51]:
#variables defined

y = boston.target
n = len(boston.target)
k = len(chosen_five)  #or p, I suppose.
y_hat = model2.predict()
y_bar = boston.target.mean()
SSR = sum([(y[i]-y_hat[i])**2 for i in range(n)])
SST = sum([(y[i]-y_bar)**2 for i in range(n)])
R_square = 1 - (SSR/SST)
R_adj = 1 - (1-R_square)*((n-1)/(n-k-1))

In [53]:
R_square

0.7421479779359903

In [54]:
R_adj

0.7395694577153502

In [52]:
print(SSR, SST)

11014.483147846442 42716.2954150198


## Level up - Optional

- Perform variable selection using forward selection, using this resource: https://planspace.org/20150423-forward_selection_with_statsmodels/. Note that this time features are added based on the adjusted-R-squared!
- Tweak the code in the `stepwise_selection()`-function written above to just perform forward selection based on the p-value.

## Summary
Great! You now performed your own feature selection methods!